<a href="https://colab.research.google.com/github/aihtn2708/Comparing-DuckDB-with-SQLite-and-Pandas/blob/main/comparing_DuckDB_vs_SQLlite_vs_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

We'll use the same dataset to demonstrate querying speed, ease of use, and functionality across DuckDB, SQLite, and Pandas.

In [ ]:
pip install duckdb

In [ ]:
import duckdb
import sqlite3
import pandas as pd
import time

[link to sample file](https://drive.google.com/file/d/10rwNjAeeJPKhk0VYWrMSAfg8xVSVaIHY/view?usp=drive_link)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Colab Notebooks/Data/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Colab Notebooks/Data


# Load Example Dataset
We'll use a CSV file for this comparison (e.g., sales_data.csv).

First, let see the result with a small dataset

In [ ]:

# Load the dataset from CSV
file_path = 'sales_data.csv'  # Ensure this file path points to your dataset
df = pd.read_csv(file_path)

# Function to time operations
def time_operation(operation):
    start_time = time.time()
    result = operation()
    elapsed_time = time.time() - start_time
    return result, elapsed_time

# Compare Operations
results = {}

# Pandas Operation
def pandas_query():
    # Example Query: Calculate average conversion rate (%) for each marketing campaign
    return df.groupby("Marketing Campaign")["Conversion Rate (%)"].mean()

results["Pandas"], time_pandas = time_operation(pandas_query)

# DuckDB Operation
duckdb_conn = duckdb.connect()
duckdb_conn.execute("CREATE TABLE sales_data AS SELECT * FROM read_csv_auto(?)", (file_path,))
def duckdb_query():
    # Example Query: Calculate average conversion rate (%) for each marketing campaign
    return duckdb_conn.execute("""
        SELECT "Marketing Campaign", AVG("Conversion Rate (%)") AS avg_conversion_rate
        FROM sales_data
        GROUP BY "Marketing Campaign"
    """).fetchdf()

results["DuckDB"], time_duckdb = time_operation(duckdb_query)

# SQLite Operation
sqlite_conn = sqlite3.connect(':memory:')
df.to_sql("sales_data", sqlite_conn, if_exists="replace", index=False)
def sqlite_query():
    # Example Query: Calculate average conversion rate (%) for each marketing campaign
    return pd.read_sql("""
        SELECT "Marketing Campaign", AVG("Conversion Rate (%)") AS avg_conversion_rate
        FROM sales_data
        GROUP BY "Marketing Campaign"
    """, sqlite_conn)

results["SQLite"], time_sqlite = time_operation(sqlite_query)

# Display Results
print("Pandas Query Time:", time_pandas)
print("DuckDB Query Time:", time_duckdb)
print("SQLite Query Time:", time_sqlite)

for key, result in results.items():
    print(f"\n{key} Results:\n", result)

Pandas Query Time: 0.011635065078735352
DuckDB Query Time: 0.011954069137573242
SQLite Query Time: 0.0018494129180908203

Pandas Results:
 Marketing Campaign
Fall Sales Campaign    17.592000
New Year Campaign      17.620253
Spring Promo 2024      16.875576
Summer Deals 2024      17.270130
Winter Promo 2023      17.187348
Name: Conversion Rate (%), dtype: float64

DuckDB Results:
     Marketing Campaign  avg_conversion_rate
0  Fall Sales Campaign            17.592000
1    Summer Deals 2024            17.270130
2    Spring Promo 2024            16.875576
3    Winter Promo 2023            17.187348
4    New Year Campaign            17.620253

SQLite Results:
     Marketing Campaign  avg_conversion_rate
0  Fall Sales Campaign            17.592000
1    New Year Campaign            17.620253
2    Spring Promo 2024            16.875576
3    Summer Deals 2024            17.270130
4    Winter Promo 2023            17.187348


# Try with **NYC Taxi Trip Data**

*   setup below is for downloading directly from kaggle



In [ ]:
Set up kaggle

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download -d elemento/nyc-yellow-taxi-trip-data

Dataset URL: https://www.kaggle.com/datasets/elemento/nyc-yellow-taxi-trip-data
License(s): U.S. Government Works
 99% 1.77G/1.78G [00:28<00:00, 54.2MB/s]
100% 1.78G/1.78G [00:28<00:00, 66.4MB/s]


In [ ]:
!unzip nyc-yellow-taxi-trip-data.zip

Archive:  nyc-yellow-taxi-trip-data.zip
  inflating: yellow_tripdata_2015-01.csv  
  inflating: yellow_tripdata_2016-01.csv  
  inflating: yellow_tripdata_2016-02.csv  
  inflating: yellow_tripdata_2016-03.csv  


# Sample data of NYC Taxi Trip Data

In [ ]:
data = pd.read_csv('yellow_tripdata_2016-03.csv')

In [ ]:
data.count()

,0
VendorID,1048575
TripID,1048575
passenger_count,1048575
trip_distance,1048575


In [ ]:
import duckdb
import sqlite3
import pandas as pd
import time

# Path to the dataset
file_path = "yellow_tripdata_2016-03.csv"

# Load dataset into Pandas
print("Loading data into Pandas...")
df = pd.read_csv(file_path)

# Function to time operations
def time_operation(operation):
    start_time = time.time()
    result = operation()
    elapsed_time = time.time() - start_time
    return result, elapsed_time

# Compare Operations
results = {}

# Pandas Operation
def pandas_query():
    # Example Query: Average trip distance per passenger count
    return df.groupby("passenger_count")["trip_distance"].mean()

results["Pandas"], time_pandas = time_operation(pandas_query)

# DuckDB Operation
print("Loading data into DuckDB...")
duckdb_conn = duckdb.connect()
duckdb_conn.execute("CREATE TABLE trips AS SELECT * FROM read_csv_auto(?)", (file_path,))

def duckdb_query():
    # Example Query: Average trip distance per passenger count
    return duckdb_conn.execute("""
        SELECT passenger_count, AVG(trip_distance) AS avg_trip_distance
        FROM trips
        GROUP BY passenger_count
    """).fetchdf()

results["DuckDB"], time_duckdb = time_operation(duckdb_query)

# SQLite Operation
print("Loading data into SQLite...")
sqlite_conn = sqlite3.connect(':memory:')
df.to_sql("trips", sqlite_conn, if_exists="replace", index=False)

def sqlite_query():
    # Example Query: Average trip distance per passenger count
    return pd.read_sql("""
        SELECT passenger_count, AVG(trip_distance) AS avg_trip_distance
        FROM trips
        GROUP BY passenger_count
    """, sqlite_conn)

results["SQLite"], time_sqlite = time_operation(sqlite_query)

# Display Results
print("\nQuery Execution Times:")
print("Pandas Query Time:", time_pandas)
print("DuckDB Query Time:", time_duckdb)
print("SQLite Query Time:", time_sqlite)

print("\nResults:")
for key, result in results.items():
    print(f"\n{key} Results:\n", result)


Loading data into Pandas...
Loading data into DuckDB...
Loading data into SQLite...

Query Execution Times:
Pandas Query Time: 0.03648972511291504
DuckDB Query Time: 0.01524209976196289
SQLite Query Time: 0.579315185546875

Results:

Pandas Results:
 passenger_count
0     2.230400
1     9.316340
2     2.997320
3     2.844738
4     2.866102
5     2.878796
6     2.793079
7    10.325000
8     0.600000
Name: trip_distance, dtype: float64

DuckDB Results:
    passenger_count  avg_trip_distance
0                0           2.230400
1                1           9.316340
2                2           2.997320
3                3           2.844738
4                4           2.866102
5                5           2.878796
6                6           2.793079
7                7          10.325000
8                8           0.600000

SQLite Results:
    passenger_count  avg_trip_distance
0                0           2.230400
1                1           9.316340
2                2           2.9

* DuckDB is highly suited for large-scale analytical tasks, especially in-memory operations and complex aggregations.
* Pandas is a good choice for small to medium datasets where memory isn't a bottleneck.
* SQLite remains a reliable lightweight database for transactional data but isn't the best tool for analytical processing on large datasets.